In [1]:
import multipsi as mtp
import numpy as np
import veloxchem as vlx

mol_str = """2

O   -0.6   0.0  0.0
O    0.6   0.0  0.0
"""
molecule = vlx.Molecule.read_xyz_string(mol_str)
basis = vlx.MolecularBasis.read(molecule, "cc-pvdz", ostream=None)
molecule.set_multiplicity(3)

rohf_drv = vlx.ScfRestrictedOpenDriver()
rohf_drv.ostream.mute()
rohf_results = rohf_drv.compute(molecule, basis)

In [2]:
molecule.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [3]:
print(f" RHF energy: {rohf_drv.get_scf_energy() : 14.8f}")

 RHF energy:  -149.60946112


In [4]:
rohf_drv.molecular_orbitals.print_orbitals(molecule, basis)

                                                                                                                          
                                                 Spin Restricted Orbitals                                                 
                                                 ------------------------                                                 
                                                                                                                          
               Molecular Orbital No.   4:                                                                                 
               --------------------------                                                                                 
               Occupation: 2.000 Energy:   -1.09511 a.u.                                                                  
               (   1 O   2s  :     0.36) (   1 O   3s  :     0.45) (   2 O   2s  :    -0.36)                              
               (

In [5]:
casscf_drv = mtp.McscfDriver()
orbital_space = mtp.OrbSpace(molecule, rohf_drv.mol_orbs)
orbital_space.cas(8, 6)

casscf_results = casscf_drv.compute(molecule, basis, orbital_space)

                                                                                                                          
                          Multi-Configurational Self-Consistent Field Driver
                                                                                                                          



               Active space definition:
               ------------------------
               Number of inactive (occupied) orbitals: 4
               Number of active orbitals:              6
               Number of virtual orbitals:             18

               This is a CASSCF wavefunction: CAS(8,6)

               CI expansion:
               -------------
               Number of determinants:      120


                                                                                                                          
               ╭────────────────────────────────────╮
               │          Driver settings           │
               ╰────────────────────────────────────╯
                                                                                                                          
                                                                                                                          
               Number of states              :   1


        1     -149.670911029     0.0e+00      4.1e-03          0        0.40    0:00:00


        2     -149.698155754    -2.7e-02      2.1e-03          0        0.40    0:00:00


        3     -149.702771606    -4.6e-03      1.7e-03          0        0.56    0:00:00


...

        8     -149.708367177    -5.2e-08      8.2e-09          0        0.56    0:00:00


        9     -149.708367181    -4.1e-09      9.7e-10          0        0.68    0:00:00
** Convergence reached in 9 iterations


       10     -149.708367181    -3.9e-10      4.4e-11          0        0.80    0:00:00
                                                                                                                          
               Final results
               -------------
                                                                                                                          
* State 1
  - S^2    : 2.00  (multiplicity = 3.0 )
  - Energy : -149.70836718142886
  - Natural orbitals
1.96300 1.96300 1.96098 1.03638 1.03638 0.04027
                                                                                                                          
                                                 Spin Restricted Orbitals                                                 
                                                 ------------------------                                                 
                                                                                      

                                                                                                                          
                                                Dipole moment for state 1                                                 
                                               ---------------------------                                                
                                                                                                                          
                                   X   :         0.000000 a.u.         0.000000 Debye                                     
                                   Y   :         0.000000 a.u.         0.000000 Debye                                     
                                   Z   :        -0.000000 a.u.        -0.000000 Debye                                     
                                 Total :         0.000000 a.u.         0.000000 Debye                                     
                

                                                                                                                          
Total MCSCF time: 00:00:02


In [6]:
print(f"CAS(8,6) energy: {casscf_results["energies"][0] : 12.8f}")

CAS(8,6) energy: -149.70836718


In [7]:
casscf_results["ci_vectors"].print(0)

Determinant     coef.    weight 
222aa0          0.972    0.945 
aa2220         -0.155    0.024 



In [8]:
print(casscf_results["natural_occupations"][0])

[1.96299851 1.96299851 1.96098176 1.03637599 1.03637599 0.04026925]


In [9]:
uhf_drv = vlx.ScfUnrestrictedDriver()
uhf_drv.ostream.mute()
uhf_results = uhf_drv.compute(molecule, basis)

In [10]:
print(f"UDFT energy: {uhf_drv.get_scf_energy() : 14.8f}")

UDFT energy:  -149.62899231


In [11]:
norb = basis.get_dimensions_of_basis()
n_alpha_electrons = 9
n_beta_electrons = 7

S = uhf_results["S"]
C_alpha = uhf_results["C_alpha"]
C_beta = uhf_results["C_beta"]

D_alpha = np.zeros((norb, norb))
for i in range(n_alpha_electrons):
    D_alpha[i, i] = 1

D_beta = np.zeros((norb, norb))
for i in range(n_beta_electrons):
    D_beta[i, i] = 1

In [12]:
S_ab = np.einsum("ap, bq, ab -> pq", C_alpha, C_beta, S)
D = D_alpha + np.einsum("pr, qs, rs -> pq", S_ab, S_ab, D_beta)

occ_numbers, U = np.linalg.eigh(D)

In [13]:
print("Orbital  Occ. number")
for orb, occ in enumerate(list(occ_numbers)[::-1]):
    if occ > 1e-3:
        print(f"{orb + 1 : >2} {occ : 12.3f}")

Orbital  Occ. number
 1        2.000
 2        2.000
 3        2.000
 4        1.999
 5        1.997
 6        1.994
 7        1.994
 8        1.000
 9        1.000
10        0.006
11        0.006
12        0.003


In [14]:
natural_orbs = uhf_drv.natural_orbitals(uhf_results)
natural_orbs.print_orbitals(molecule, basis)

                                                                                                                          
                                                 Spin Restricted Orbitals                                                 
                                                 ------------------------                                                 
                                                                                                                          
               Molecular Orbital No.   4:                                                                                 
               --------------------------                                                                                 
               Occupation: 3.998 Energy:   -1.34641 a.u.                                                                  
               (   1 O   2s  :     0.33) (   1 O   3s  :     0.37) (   1 O   1p+1:    -0.16)                              
               (